In [36]:
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from typing import TypedDict
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="qwen-2.5-32b")

class State(TypedDict):
    message: str
    story: str

def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

tools = [add]

llm_with_tool = llm.bind_tools(tools)

def assistant(state: State):
    response = llm_with_tool.invoke(state['message'])
    return {"message": state['message'], "story": response.content}

from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(State)

graph_builder.add_node("assistant", assistant)
graph_builder.add_node("tools", ToolNode(tools))

graph_builder.add_edge(START, "assistant")
graph_builder.add_conditional_edges("assistant", tools_condition)
graph_builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = graph_builder.compile(interrupt_before=["tools"], checkpointer=memory)

thread = {"configurable": {"thread_id": "1"}}

initial_input = "what is Gen AI"
for event in graph.stream({"message": initial_input, "story": ""}, thread, stream_mode="values"):
    print(event)

{'message': 'what is Gen AI', 'story': ''}


ValueError: No messages found in input state to tool_edge: {'message': 'what is Gen AI', 'story': 'Gen AI, short for Generative Artificial Intelligence, refers to a category of AI technologies designed to create new digital content that is original or unique. This can include text, images, music, and more. Generative AI models are often trained on large datasets and use techniques like deep learning to generate content that is similar to what humans create. These systems can be used in various applications, such as writing articles, composing music, or generating visual art. They represent a significant advancement in how machines can imitate and even surpass human creativity in certain areas.'}

In [ ]:
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, List, Dict, Any
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="qwen-2.5-32b")

class State(TypedDict):
    messages: List[Dict[str, Any]]  # List of messages exchanged
    story: str  # Additional state for the story

def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

tools = [add]

llm_with_tool = llm.bind_tools(tools)

def assistant(state: State):
    # Get the last message from the conversation
    last_message = state['messages'][-1]['content']
    # Invoke the LLM with the last message
    response = llm_with_tool.invoke(last_message)
    # Update the state with the response
    state['messages'].append({"role": "assistant", "content": response.content})
    return state

from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(State)

graph_builder.add_node("assistant", assistant)
graph_builder.add_node("tools", ToolNode(tools))

graph_builder.add_edge(START, "assistant")
graph_builder.add_conditional_edges("assistant", tools_condition)
graph_builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = graph_builder.compile(interrupt_before=["tools"], checkpointer=memory)

thread = {"configurable": {"thread_id": "1"}}

# Initialize the state with a user message
initial_input = "what is 2+3?"
initial_state = {
    "messages": [{"role": "user", "content": initial_input}],
    "story": ""
}

# Stream the graph execution
for event in graph.stream(initial_state, thread, stream_mode="values"):
    print(event['messages'][-1])

SyntaxError: invalid syntax (3923413989.py, line 63)

In [42]:
state=graph.get_state(thread)
state.next

()